In [1]:
library(purrr)
library(tidyverse)
library(Seurat)
library(ggplot2)
library(scales)

Warning message in system("timedatectl", intern = TRUE):
"running command 'timedatectl' had status 1"
-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.5     v dplyr   1.0.7
v tibble  3.1.6     v stringr 1.4.0
v tidyr   1.1.4     v forcats 0.5.1
v readr   2.1.0     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'spatstat.geom':
  method     from
  print.boxx cli 

Attaching SeuratObject


Attaching package: 'scales'


The following object is masked from 'package:readr':

    col_factor


The following object is masked from 'package:purrr':

    discard




In [2]:
source('paths.R')
source('misc.R')
source('factors.R')

In [3]:
p_from_odds <- function(o) o/(1+o)
odds_from_p <- function(p) p/(1-p)

# 1. Load Data

In [4]:
m <- load_object(path_at('out', 'cache')('annotated', 'metadata', 'Robj')) %>% filter(!is.na(cell_type))

# 2. Prepare data

## 2.1 VLC counts

In [5]:
vlc_counts_c <- (
    m
    %>% group_by(condition_short, cell_type, viral_load_class) %>% count(name='n_class')
    %>% mutate(vlc_c = viral_load_class %>% fct_collapse(`Insignificant`=c('Zero', 'Noise')))
    %>% group_by(cell_type, condition_short, vlc_c)
    %>% summarize(n_class = sum(n_class))
    %>% ungroup()
)
vlc_counts_c %>% head()

`summarise()` has grouped output by 'cell_type', 'condition_short'. You can override using the `.groups` argument.



cell_type,condition_short,vlc_c,n_class
<ord>,<ord>,<ord>,<int>
Basal,M,Insignificant,558
Basal,V,Medium,562
Basal,V,High,17
Basal,VCy,Insignificant,133
Basal,VCy,Low,950
Basal,VCy,Medium,68


## 2.2 Expand to have all possible combinations

In [6]:
vlc_counts_c_full <- (
    vlc_counts_c
    %>% expand(cell_type, condition_short, vlc_c)
    %>% left_join(vlc_counts_c)
    %>% mutate(n_class = n_class %>% replace_na(0))
)

vlc_counts_c_full %>% head()

Joining, by = c("cell_type", "condition_short", "vlc_c")



cell_type,condition_short,vlc_c,n_class
<ord>,<ord>,<ord>,<dbl>
Basal,M,Insignificant,558
Basal,M,Low,0
Basal,M,Medium,0
Basal,M,High,0
Basal,V,Insignificant,0
Basal,V,Low,0


In [7]:
tabs <- (
    vlc_counts_c_full
    %>% rename(category = vlc_c)
    %>% pivot_wider(names_from=category, values_from=n_class)
)
tabs %>% head()

cell_type,condition_short,Insignificant,Low,Medium,High
<ord>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>
Basal,M,558,0,0,0
Basal,V,0,0,562,17
Basal,VCy,133,950,68,2
Basal,VIf,206,456,7,1
Basal,VCyIf,799,4,0,0
Basal,MCyIf,750,0,0,0


In [8]:
.chisq_test_matrix <- function(m) {
    if (1 == ncol(m) | 1 == nrow(m)) return(NA)
    r <- chisq.test(m, simulate.p.value = TRUE, B=100000)
    return(r$p.value)
}

chisq_test <- function(data) (
    data
    %>% select_if(is.numeric)
    %>% select(where(~ any(. != 0)))
    %>% as.matrix()
    %>% .chisq_test_matrix()
)

# 3. Test Across Conditions

In [9]:
condition_pairs_selectors <- (
    c('V_vs_M', 'VCy_vs_V', 'VIf_vs_V', 'VCyIf_vs_V', 'VCy_vs_VIf',
      'VCyIf_vs_VCy', 'VCyIf_vs_VIf', 'MCyIf_vs_M', 'VCyIf_vs_M')
    %>% set_names()
    %>% map(~str_split(.x, '_vs_')[[1]])
    %>% map(~function(df)(df %>% filter(condition_short %in% .x)))
)

test_condition_pairs <- function(tbl)(
    condition_pairs_selectors
    %>% map(~.x(tbl))
    %>% map(chisq_test)
)
    
p_vals <- (
    cell_type_levels
    %>% set_names()
    %>% map(~tabs %>% filter(cell_type == .x))
    %>% map(test_condition_pairs)
    %>% bind_rows(.id='contrast')
    %>% mutate_if(is.numeric, scales::pvalue_format(accuracy=1e-3))
)
p_vals

contrast,V_vs_M,VCy_vs_V,VIf_vs_V,VCyIf_vs_V,VCy_vs_VIf,VCyIf_vs_VCy,VCyIf_vs_VIf,MCyIf_vs_M,VCyIf_vs_M
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Basal,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,NA,0.149
BdiS,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,>0.999,0.612
Secretory,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,0.461,>0.999
Ciliated,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,<0.001,0.615,0.005


# 4. Test across cell types

In [10]:
cell_type_pairs_selectors <- (
    c('BdiS_vs_Basal', 'Secretory_vs_BdiS', 'Ciliated_vs_Secretory')
    %>% set_names()
    %>% map(~str_split(.x, '_vs_')[[1]])
    %>% map(~function(df)(df %>% filter(cell_type %in% .x)))
)
    
test_cell_type_pairs <- function(tbl)(
    cell_type_pairs_selectors
    %>% map(~.x(tbl))
    %>% map(chisq_test)
)
    
p_vals <- (
    conditions_meta$condition_short
    %>% set_names()
    %>% map(~tabs %>% filter(condition_short == .x))
    %>% map(test_cell_type_pairs)
    %>% bind_rows(.id='condition_short')
    %>% mutate_if(is.numeric, scales::pvalue_format(accuracy=1e-3))
)
p_vals

condition_short,BdiS_vs_Basal,Secretory_vs_BdiS,Ciliated_vs_Secretory
<chr>,<chr>,<chr>,<chr>
M,0.288,>0.999,0.606
V,<0.001,<0.001,<0.001
VCy,<0.001,0.005,<0.001
VIf,0.117,0.502,<0.001
VCyIf,0.611,0.122,<0.001
MCyIf,0.306,0.380,0.451
